# 2020 SUMMER ESC :: Week 2 세션 과제

2020.08.02 김채형

### Q1-1)

아래에 주어진 주석을 기반으로 하여 코딩을 해주세요.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [2]:
# 파라미터 설정
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [3]:
# data
## train set과 test set으로 나누어 MNIST data 불러오기
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/Users/distiller/project/conda/conda-bld/pytorch_1591914925853/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [4]:
# data loader
## dataset loader에 train과 test 할당하기 (batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)

In [5]:
# nn layers
## Layer 쌓기
## 조건 : 3개의 Layer 사용 / DropOut 사용 (p=0.3) / ReLU 함수 사용 / Batch normalization 하기  
## 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100), 3rd Layer(100,10)
linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)
bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)

In [6]:
# weight initialization : xavier initialization
## xavier initialization을 이용하여 각 layer의 weight 초기화
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-3.8792e-02, -1.5501e-01,  1.2453e-01,  1.4407e-01, -2.0475e-01,
         -1.5531e-01,  1.8045e-01, -1.8666e-01, -6.5143e-02, -8.0612e-02,
         -1.9018e-01, -2.8504e-03, -1.6596e-01, -1.6611e-01,  8.5252e-02,
         -1.9500e-01, -6.9105e-02,  7.5690e-02,  2.0679e-01,  1.3518e-01,
         -1.3819e-01,  4.2981e-02, -2.2658e-01,  1.6310e-01,  5.9000e-02,
          1.2462e-01, -2.2421e-01,  1.3168e-01, -3.1367e-02,  1.6436e-01,
         -2.3227e-01,  1.6357e-01,  1.0269e-01, -1.6034e-01,  2.0724e-01,
          1.7530e-01,  7.5835e-02,  6.8787e-02, -1.6235e-01, -1.0325e-01,
          1.7413e-02,  1.4418e-01, -2.2946e-01,  6.3656e-02,  1.2483e-01,
         -1.3916e-01,  1.2382e-01,  1.0330e-01, -1.7153e-02,  1.0329e-01,
         -1.8718e-01,  1.5454e-01, -2.0483e-01, -1.2387e-02,  6.7946e-03,
         -2.0540e-01,  8.1489e-02,  1.5138e-01, -1.0487e-01, -7.2547e-02,
          2.1172e-01,  2.2560e-01,  1.7233e-01,  6.5866e-02,  8.2770e-03,
         -1.3090

In [7]:
# model
## torch.nn.Sequential을 이용하여 model 정의하기
## 쌓는 순서: linear - Batch Normalization Layer - ReLU - DropOut
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3)

In [8]:
# Loss function & optimizer
## Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()
## optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop

## cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

## Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것)
for epoch in range(training_epochs):
    
    model.train()
    avg_cost = 0
        
    ## train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        
        # X, Y reshape
        X = X.view(-1, 28*28)
        Y = Y
        
        optimizer.zero_grad() # 0으로 초기화
        
        # H(x) 계산
        prediction = model(X)
        
        # Cost 계산
        loss = criterion(prediction, Y)
        
        # Cost 통해 H(x) 개선
        loss.backward()
        optimizer.step()
    
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')

Epoch: 0001 cost = 0.512693763
Epoch: 0002 cost = 0.379160732
Epoch: 0003 cost = 0.328012943
Epoch: 0004 cost = 0.305955201
Epoch: 0005 cost = 0.295698673
Epoch: 0006 cost = 0.285342395
Epoch: 0007 cost = 0.279721856


In [ ]:
# Test the model
## test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
## X_test 불러올 때 view를 사용하여 차원 변환할 것 / Y_test를 불러올때 labels 사용
## accuracy의 초기 값 설정(0으로) 꼭 할 것
with torch.no_grad(): 
    
    model.eval()
    
    accuracy = 0
    
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()

    print("Accuracy: ", accuracy.item())
    
    ## Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

### Q1-2)

지금까지는 Layer의 수를 바꾸거나, Batch Normalization Layer를 추가하는 등 Layer에만 변화를 주며 모델의 성능을 향상 시켰습니다.  
이번 문제에서는 위에서 만든 모델에서 있던 Layer 들의 Hidden node 수를 증가 또는 감소 (ex: 200, 300, 50...) 시켰을 때, train set에서의 cost와 test set에서 Accuracy가 기존 결과와 비교하였을 때 어떻게 달라졌는지 비교해주시면 됩니다.

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

**hidden node = 75**

In [ ]:
# 파라미터 설정
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# data
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=False)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False, transform=transforms.ToTensor(), download=False)

# data loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# nn layers
linear1 = torch.nn.Linear(784, 75, bias=True)
linear2 = torch.nn.Linear(75, 75, bias=True)
linear3 = torch.nn.Linear(75, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)
bn1 = torch.nn.BatchNorm1d(75)
bn2 = torch.nn.BatchNorm1d(75)

# weight initialization : xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

# model
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3)

# Loss function & Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_total_batch = len(train_loader)
for epoch in range(training_epochs):
    
    model.train() # For batch normalization
    avg_cost = 0

    for X, Y in train_loader:
        
        # X, Y reshape
        X = X.view(-1, 28*28)
        Y = Y
        
        optimizer.zero_grad() # 0으로 초기화
        
        # H(x) 계산
        prediction = model(X)
        
        # Cost 계산
        loss = criterion(prediction, Y)
        
        # Cost 통해 H(x) 개선
        loss.backward()
        optimizer.step()
    
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')
print('\n')
# Test the model using test sets
with torch.no_grad(): 
    
    model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

**hidden node = 50**

In [ ]:
# 파라미터 설정
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# data
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=False)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False, transform=transforms.ToTensor(), download=False)

# data loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# nn layers
linear1 = torch.nn.Linear(784, 50, bias=True)
linear2 = torch.nn.Linear(50, 50, bias=True)
linear3 = torch.nn.Linear(50, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)
bn1 = torch.nn.BatchNorm1d(50)
bn2 = torch.nn.BatchNorm1d(50)

# weight initialization : xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

# model
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3)

# Loss function & Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_total_batch = len(train_loader)
for epoch in range(training_epochs):
    
    model.train()
    
    avg_cost = 0
    for X, Y in train_loader:
        
        # X, Y reshape
        X = X.view(-1, 28*28)
        Y = Y
        
        optimizer.zero_grad() # 초기화
        
        # H(x) 계산
        prediction = model(X)
        
        # Cost 계산
        loss = criterion(prediction, Y)
        
        # Cost 통해 H(x) 개선
        loss.backward()
        optimizer.step()
    
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')
print('\n')

# Test the model
with torch.no_grad(): 
    
    model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

**hidden node = 25**

In [ ]:
# parameter
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# data
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=False)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False, transform=transforms.ToTensor(), download=False)

# data loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# nn layers
linear1 = torch.nn.Linear(784, 25, bias=True)
linear2 = torch.nn.Linear(25, 25, bias=True)
linear3 = torch.nn.Linear(25, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)
bn1 = torch.nn.BatchNorm1d(25)
bn2 = torch.nn.BatchNorm1d(25)

# weight initialization : xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

# model
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3)

# Loss function & Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_total_batch = len(train_loader)
for epoch in range(training_epochs):
    
    model.train()
    
    avg_cost = 0
    for X, Y in train_loader:
        
        # X, Y reshape
        X = X.view(-1, 28*28)
        Y = Y
        
        optimizer.zero_grad() # 0으로 초기화
        
        # H(x) 계산
        prediction = model(X)
        
        # Cost 계산
        loss = criterion(prediction, Y)
        
        # Cost 통해 H(x) 개선
        loss.backward()
        optimizer.step()
    
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')
print('\n')

# Test the model
with torch.no_grad(): 
    
    model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

**Accuracy 결과**

|hidden node 개수|Cost|Accuracy|
|---------------|----|--------|
|25|0.495|0.836|
|50|0.325|0.935|
|75|0.280|0.930|
|100|0.238|0.932|

In [ ]:
"""
# parameter
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# data
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=False)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False, transform=transforms.ToTensor(), download=False)

# data loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

hidden_nodes = [20, 60, 100, 140, 180, 220, 260, 300]
acc_list = []
for hn in hidden_nodes:
    
    # nn layers
    linear1 = torch.nn.Linear(784, hn, bias=True)
    linear2 = torch.nn.Linear(hn, hn, bias=True)
    linear3 = torch.nn.Linear(hn, 10, bias=True)
    relu = torch.nn.ReLU()
    dropout = torch.nn.Dropout(p=0.3)
    bn1 = torch.nn.BatchNorm1d(hn)
    bn2 = torch.nn.BatchNorm1d(hn)
    
    # weight initialization : xavier initialization
    torch.nn.init.xavier_uniform_(linear1.weight)
    torch.nn.init.xavier_uniform_(linear2.weight)
    torch.nn.init.xavier_uniform_(linear3.weight)
    
    # model
    model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                                linear2, bn2, relu, dropout,
                                linear3)

    # Loss function & Optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    train_total_batch = len(train_loader)
    for epoch in range(training_epochs):
        
        model.train()
        
        avg_cost = 0
        for X, Y in train_loader:
            
            # X, Y reshape
            X = X.view(-1, 28*28)
            Y = Y
            
            optimizer.zero_grad()

            prediction = model(X)
            loss = criterion(prediction, Y)
            loss.backward()
            optimizer.step()
            
            avg_cost += loss / train_total_batch
            
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        
        print('Learning finished')
        print('\n')
    
    # Test the model
    with torch.no_grad(): 
        
        model.eval()
        X_test = mnist_test.test_data.view(-1, 28*28).float()
        Y_test = mnist_test.test_labels
        
        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        acc_list.append(accuracy)
        print("Accuracy: ", accuracy.item())
"""